In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
base_dir = '/content/drive/My Drive/Deep Learning/Natural Face'


train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')
BATCH_SIZE = 32
IMG_RES = (160, 160)

training_set = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_RES)
validation_set = image_dataset_from_directory(validation_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_RES)

Found 10343 files belonging to 7 classes.
Found 4424 files belonging to 7 classes.


In [ ]:
num_training_examples = 10343
num_classes = 7

In [ ]:
train_image_generator = ImageDataGenerator(
      rescale = 1./255,
	  rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_image_generator = ImageDataGenerator(
      rescale = 1./255,
	  rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [ ]:
def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label
IMAGE_RES = 224
train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                           directory=train_dir,
                                                           target_size=(IMAGE_RES,IMAGE_RES), 
                                                           shuffle=True,
                                                           class_mode='sparse')
val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                              directory=validation_dir,
                                                              shuffle=False,
                                                              target_size=(IMAGE_RES,IMAGE_RES), 
                                                              class_mode='sparse')

Found 10343 images belonging to 7 classes.
Found 4424 images belonging to 7 classes.


In [ ]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES, 3))

In [ ]:
feature_extractor.trainable = True

In [ ]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(num_classes)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 7)                 8967      
Total params: 2,266,951
Trainable params: 2,232,839
Non-trainable params: 34,112
_________________________________________________________________


In [ ]:
from keras import backend as K

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy',precision_m])

EPOCHS = 5

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(BATCH_SIZE)))
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
324/324 [==============================] - 3719s 11s/step - loss: 1.6474 - accuracy: 0.4142 - precision_m: 1.2004 - val_loss: 1.4788 - val_accuracy: 0.5264 - val_precision_m: 1.0036
Epoch 2/5
324/324 [==============================] - 2387s 7s/step - loss: 1.1970 - accuracy: 0.5945 - precision_m: 1.0522 - val_loss: 1.3508 - val_accuracy: 0.5669 - val_precision_m: 1.0154
Epoch 3/5
324/324 [==============================] - 2395s 7s/step - loss: 1.0365 - accuracy: 0.6603 - precision_m: 1.0192 - val_loss: 1.2633 - val_accuracy: 0.5961 - val_precision_m: 1.0065
Epoch 4/5
324/324 [==============================] - 2427s 7s/step - loss: 0.9779 - accuracy: 0.6842 - precision_m: 1.0233 - val_loss: 1.4704 - val_accuracy: 0.5588 - val_precision_m: 1.0032
Epoch 5/5
324/324 [==============================] - 2342s 7s/step - loss: 0.9201 - accuracy: 0.7069 - precision_m: 1.0083 - val_loss: 1.2423 - val_accuracy: 0.6112 - val_precision_m: 1.0023
